In [1]:
!pip install tensorflow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the Saved Model

In [3]:
from tensorflow.keras.models import load_model

model_path = "/content/drive/MyDrive/Violence Detection/violence_mobilenetV2_FINAL.keras"
model = load_model(model_path)

print("✔ Model loaded successfully and ready for inference.")


✔ Model loaded successfully and ready for inference.


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 38 variables whereas the saved optimizer has 74 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# Add Frame Extraction Function

(Needed to preprocess video exactly like training)

In [4]:
import cv2
import numpy as np

IMG_SIZE = 112
FRAMES_PER_VIDEO = 5

def extract_frames(video_path, num_frames=FRAMES_PER_VIDEO, img_size=IMG_SIZE):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames <= 0:
        cap.release()
        return np.array([])

    frame_indices = np.linspace(0, total_frames - 1, num_frames).astype(int)
    frames = []

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (img_size, img_size))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = frame / 255.0
            frames.append(frame)

    cap.release()
    return np.array(frames)


# Add Prediction Function

In [5]:
def classify_video(video_path):
    frames = extract_frames(video_path)

    if frames.size == 0:
        return "❌ Unable to read video or no frames extracted."

    preds = model.predict(frames)
    probability = preds.mean()

    print(f"\n📊 Violence Probability: {probability:.4f}")

    if probability >= 0.5:
        return "🟥 RESULT: VIOLENCE DETECTED"
    else:
        return "🟩 RESULT: NON-VIOLENCE"


# Upload a Test Video

In [10]:
from google.colab import files

uploaded = files.upload()
video_file = list(uploaded.keys())[0]
video_path = "/content/" + video_file

print("Uploaded video:", video_path)


Saving V_29.mp4 to V_29.mp4
Uploaded video: /content/V_29.mp4


# Run Classification

In [11]:
result = classify_video(video_path)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

📊 Violence Probability: 0.9955
🟥 RESULT: VIOLENCE DETECTED
